In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier

In [2]:
base_dir = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/raw_data"

dfs = []
for i in range(1, 43):
    poteaux_path = os.path.join(base_dir, f"maquette{i}", f"poteaux{i}.csv")
    if os.path.exists(poteaux_path):
        # Cherche la ligne d'en-tête réelle
        with open(poteaux_path, encoding="utf-8") as f:
            for idx, line in enumerate(f):
                if line.startswith("Id;"):
                    header_row = idx
                    break
        try:
            df = pd.read_csv(poteaux_path, sep=';', header=header_row)
            dfs.append(df)
            print(f"Chargé : {poteaux_path} ({df.shape[0]} lignes, {df.shape[1]} colonnes)")
            print(df.head())
        except Exception as e:
            print(f"Erreur de parsing : {poteaux_path} -> {e}")
    else:
        print(f"Fichier non trouvé : {poteaux_path}")

if dfs:
    poteaux_concat = pd.concat(dfs, ignore_index=True)
    print(f"Total concaténé : {poteaux_concat.shape[0]} lignes, {poteaux_concat.shape[1]} colonnes")
else:
    poteaux_concat = pd.DataFrame()
    print("Aucun fichier murs.csv trouvé.")

poteaux_concat.head()

Chargé : /Users/fabiancordenod/code/fqbq69/BIMpredict-/raw_data/maquette1/poteaux1.csv (215 lignes, 110 colonnes)
       Id 011EC_Lot         012EC_Ouvrage 013EC_Localisation  \
0  846158        GO  POTEAU RECTANGULAIRE                NaN   
1  846164        GO  POTEAU RECTANGULAIRE                NaN   
2  846166        GO  POTEAU RECTANGULAIRE                NaN   
3  846168        GO  POTEAU RECTANGULAIRE                NaN   
4  846170        GO  POTEAU RECTANGULAIRE                NaN   

  014EC_Mode Constructif     Nom    Image  Style de poteau         Catégorie  \
0                 BANCHE  110x30  <Aucun>                0  Poteaux porteurs   
1                 BANCHE  110x30  <Aucun>                0  Poteaux porteurs   
2                 BANCHE  110x30  <Aucun>                0  Poteaux porteurs   
3                 BANCHE  110x30  <Aucun>                0  Poteaux porteurs   
4                 BANCHE  110x30  <Aucun>                0  Poteaux porteurs   

   Type prédéfini d'

,Id,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Nom,Image,Style de poteau,Catégorie,Type prédéfini d'IFC,...,Classe metal,Prefixe,DIAMETRE,LARGEUR,LONGUEUR,Décalage par rapport à l'hôte,Hôte,STR_NIV_ref,b poteau,h poteau
0,846158,GO,POTEAU RECTANGULAIRE,NaN,BANCHE,110x30,<Aucun>,0.0,Poteaux porteurs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,846164,GO,POTEAU RECTANGULAIRE,NaN,BANCHE,110x30,<Aucun>,0.0,Poteaux porteurs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,846166,GO,POTEAU RECTANGULAIRE,NaN,BANCHE,110x30,<Aucun>,0.0,Poteaux porteurs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,846168,GO,POTEAU RECTANGULAIRE,NaN,BANCHE,110x30,<Aucun>,0.0,Poteaux porteurs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,846170,GO,POTEAU RECTANGULAIRE,NaN,BANCHE,110x30,<Aucun>,0.0,Poteaux porteurs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
poteaux_concat.shape

(10439, 416)

In [4]:
colonnes_a_garder = [
    "011EC_Lot",
    "012EC_Ouvrage",
    "013EC_Localisation",
    "014EC_Mode Constructif",
    "Epaisseur",
    "Sols en intersection",
    "Sols coupés (u)",
    "Sols coupants (u)",
    "Sol au-dessus",
    "Sol en-dessous",
    "Fenêtres",
    "Portes",
    "Ouvertures",
    "Murs imbriqués",
    "Mur multicouche",
    "Profil modifié",
    "Extension inférieure",
    "Extension supérieure",
    "Partie inférieure attachée",
    "Partie supérieure attachée",
    "Décalage supérieur",
    "Décalage inférieur",
    "Matériau structurel",
    "Famille et type"
]

# On ne garde que les colonnes présentes dans le DataFrame
poteaux_concat = poteaux_concat[[col for col in colonnes_a_garder if col in poteaux_concat.columns]]

In [5]:
import unicodedata
import re

def clean_col(col):
    # Enlever accents
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    # Remplacer espaces et tirets par _
    col = re.sub(r"[ \-\(\)]", "_", col)
    # Supprimer tout caractère non alphanumérique ou _
    col = re.sub(r"[^a-z0-9_]", "", col)
    # Nettoyer les doubles/triples underscores
    col = re.sub(r"_+", "_", col)
    # Supprimer _ en début/fin
    col = col.strip("_")
    return col

poteaux_concat.columns = [clean_col(c) for c in poteaux_concat.columns]
print(poteaux_concat.columns.tolist())

['011ec_lot', '012ec_ouvrage', '013ec_localisation', '014ec_mode_constructif', 'sols_en_intersection', 'sols_coupes_u', 'sols_coupants_u', 'extension_inferieure', 'extension_superieure', 'partie_inferieure_attachee', 'partie_superieure_attachee', 'decalage_superieur', 'decalage_inferieur', 'materiau_structurel', 'famille_et_type']


In [6]:
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Garder seulement les targets présents dans le DataFrame
targets_in_df = [col for col in targets if col in poteaux_concat.columns]

if not targets_in_df:
    raise ValueError(f"Aucune colonne cible trouvée dans poteaux_concat. Colonnes disponibles : {poteaux_concat.columns.tolist()}")

# X et y_multi
X = poteaux_concat.drop(columns=targets_in_df)
y_multi = poteaux_concat[targets_in_df]

if X.shape[1] == 0:
    raise ValueError("Aucune variable explicative disponible après suppression des cibles. Vérifiez vos colonnes.")

num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_features)
    ]
)

# Pipeline complet avec MultiOutputClassifier et KNN
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=10)))
])

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_multi, test_size=0.2, random_state=42)

# Supprimer les lignes avec NaN dans les targets (train et test)
train_notna = y_train.notna().all(axis=1)
test_notna = y_test.notna().all(axis=1)
X_train, y_train = X_train[train_notna], y_train[train_notna]
X_test, y_test = X_test[test_notna], y_test[test_notna]

# Entraînement
pipeline.fit(X_train, y_train)

# Prédiction et score
y_pred = pipeline.predict(X_test)
print("Accuracy moyenne multi-label :", (y_pred == y_test.values).mean())

Accuracy moyenne multi-label : 0.9700395703787451


In [7]:
print("Accuracy calculée sur", len(y_test), "échantillons.")

Accuracy calculée sur 1769 échantillons.


In [8]:
from sklearn.metrics import f1_score

# Calculer le F1-score pour chaque colonne (target) séparément
f1_micro_list = []
f1_macro_list = []
for i, col in enumerate(y_test.columns):
	f1_micro_list.append(f1_score(y_test.iloc[:, i], y_pred[:, i], average='micro'))
	f1_macro_list.append(f1_score(y_test.iloc[:, i], y_pred[:, i], average='macro'))

# Moyenne des scores F1 sur toutes les cibles
f1_micro_mean = np.mean(f1_micro_list)
f1_macro_mean = np.mean(f1_macro_list)

print(f"F1 micro (moyenne par cible): {f1_micro_mean:.4f}")
print(f"F1 macro (moyenne par cible): {f1_macro_mean:.4f}")

F1 micro (moyenne par cible): 0.9700
F1 macro (moyenne par cible): 0.7856


In [9]:
import pandas as pd
from sklearn.metrics import f1_score

# Calculer le F1-score pour chaque colonne (target) séparément
f1_per_label = []
for i, col in enumerate(y_test.columns):
    f1_per_label.append(f1_score(y_test.iloc[:, i], y_pred[:, i], average='macro'))

# Récupérer les noms de labels
label_names = list(y_test.columns)

# Construire un DataFrame pour lecture facile
f1_report = pd.DataFrame({
    'Label': label_names,
    'F1-score': f1_per_label
})

# Trier du pire au meilleur
f1_report = f1_report.sort_values(by='F1-score')

# Afficher
print(f1_report)

                    Label  F1-score
1           012ec_ouvrage  0.590631
2      013ec_localisation  0.775152
3  014ec_mode_constructif  0.786690
0               011ec_lot  0.989808


In [10]:
from sklearn.metrics import classification_report

# Classification report pour chaque cible séparément
for i, col in enumerate(y_test.columns):
	print(f"=== Classification Report for {col} ===")
	print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

=== Classification Report for 011ec_lot ===
              precision    recall  f1-score   support

      FACADE       1.00      1.00      1.00         3
          GO       1.00      1.00      1.00      1714
  METALLIQUE       1.00      0.94      0.97        52

    accuracy                           1.00      1769
   macro avg       1.00      0.98      0.99      1769
weighted avg       1.00      1.00      1.00      1769

=== Classification Report for 012ec_ouvrage ===
                      precision    recall  f1-score   support

                 CES       1.00      0.94      0.97        52
          GROS BETON       1.00      1.00      1.00         4
              LIBAGE       0.00      0.00      0.00         1
                 MUR       0.00      0.00      0.00         6
        POTEAU ANGLE       1.00      0.09      0.17        22
   POTEAU CIRCULAIRE       0.91      0.92      0.92        79
  POTEAU PARTICULIER       0.90      0.84      0.87        31
POTEAU RECTANGULAIRE       0.9

/Users/fabiancordenod/.pyenv/versions/3.10.6/envs/BIMpredict2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fabiancordenod/.pyenv/versions/3.10.6/envs/BIMpredict2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fabiancordenod/.pyenv/versions/3.10.6/envs/BIMpredict2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa